# Studying the Two Running Games in Super Bowl LVII

_Note: You can consult the solution for this live training in the file browser (`notebook_solution.ipynb`)_

Every February millions of people, both inside and outside of North America, tune into the National Football League's Super Bowl, which crowns the world champion in American football.
In football there are two ways to advance the football while on offense; running and passing.
During this training, you will learn how to use data to see which players to watch out for during Super Bowl LVII in the running game, the more unheralded aspect of a football game.
We will show you how to obtain NFL play-by-play data in Python and then use exploratory data analysis and linear models to show which players will be the ones to watch during Super Bowl LVII. 

### Obtaining data and loading packages

While we use Python in this tutorial, the `nflfastR` play-by-play datatset was developed initially in R, and the package's [help page](https://www.nflfastr.com/articles/field_descriptions.html) that gives the best description of the metadata.
You can obtain data for any year using `import_pbp_data()` from the `nfl_data_py` package and we included this code commented out for your future reference.
However, we pre-stage the data for this live training as a csv file to optimize your learning experience.

First, load the required python packages.
Use `pandas` (alias `pd`) and `numpy` (alias `np`) for data.
Use `seaborn` (alias `sns`) and `matplotlib.pyplot` (alias `plt`) for plotting.
Use `statsmodels.formula.api` alias `smf` for linear models.

In [1]:
# import required packages
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

**Note:** If you wanted to import the data for other seasons, you could use the code Eric and Richard used to obtain the original data:

```python
# code we used to obtain data
import nfl_data_py as nfl
pbp = nfl.import_pbp_data([2022])
# we only selected the columns we needed for today
pbp[['play_type', 'posteam', 'rushing_yards', 'rusher_id',
     'rusher_player_id', 'rusher_player_name', 'ydstogo', 'down',
     'yardline_100', 'run_location', 'score_differential',
     'game_seconds_remaining']].to_csv("pbp_2022.csv", index=False)
```

Now, you can load the _play-by-play_ (_pbp_, Python object `pbp`) data you'll use today using `pd.read_csv()` with `pbp_2022.csv`:

In [2]:
# load play-by-play data 
pbp = pd.read_csv('pbp_2022.csv')

Next, peak at the top of `pbp` using the `.head()`

In [3]:
# peak at head of pbp data

## Filtering Data to Rushing Plays

First, the `pbp` data needs to be filtered cleaned.
Use `query()` to preform the following:

* Save the new data frame as `pbp_run` because this is only the run play-by-play (_pbp_) data
* Filter (or, in `pandas` lingo, `query()`) to rushing plays with `play_type == "run"`
* Remove missing values for rushing_yards using `rushing_yards.notnull()`
* Remove missing values for rusher_id `rusher_id.notnull()`
* Rest the data frame's index, and 
* Look at the header of the data frame

Remember, use `&` to merge multiple filter criteria.
After filtering, look at the `head()` of the data:

In [9]:
# query pbp


# peak at head of data

## Who Are the Best Rushers in Sunday's Game?

Next, you will examine which players are the best at rushing the football for the players will be in the game on Sunday.
First, select the data for the two teams in the Super Bowl, the Philadelphia Eagles (`PHI`) and the Kansas City Chiefs (`KC`) using the `isin` function from `pandas` with the Team of Possession (which team has the ball; `posteam`) column. To this,

1. Create a list of Super Bowl teams, `sb_teams`
2. Use the `.loc` command with `pbp_run` and the `.isin()` function on `posteam` column (**Hint:**, this looks like `pbp_run['posteam'].isin(sb_teams)`).
3. `reset_index()` on the new data frame
4. Save the outputs as `pbp_run_sb`

In [12]:
# create a list of teams in the Super Bowl

# Filter out only the Super Bowl teams using `isin()`


Next, check the data to make sure it is correct and you only have these two teams by using the `posteam` column and looking for the `unique()` values in the column:

In [13]:
# Look at the unique posteam values in `pbp_run_sb`


Now, you can aggregate over the whole season for each player's rushing yards (sum of `rushing_yards`) and yards-per-carry (mean of `rushing yards`).
For this, "group by" the `posteam` so we know which team the player belongs to, and both `rusher_player_id` and `rusher_player_name` because some players have the same name.
Save this operation as `pbp_run_sb_yards`:

In [14]:
# 1. use pbp_run_sb
# 2. groupby ["posteam", "rusher_player_id", "rusher_player_name"]
# 3. then aggregate `{"rushing_yards": ["sum", "mean", "count"]}`
# 4. save ouput as pbp_run_sb_yards


Next, get the `pbp_run_sb_yards` data ready to examine with two steps:
    
1. Filer using `.loc[]` with `pbp_run_sb_yards[('rushing_yards', 'count')] > 20` to look at only players with more than 20 rushing plays
2. Sort by the player's total rushing yards (`('rushing_yards', 'sum')`) with `ascending=[False]` to help us see who had the most yards

In [17]:
# 1. Filter with pbp_run_sb_yards[('rushing_yards', 'count')] > 20]
# 2. Look at the results by sorting by `('rushing_yards', 'sum')`

Notice also that the leading rusher for the Chiefs is Isiah Pacheco, who is a seventh-round pick in the most-recent NFL draft out of Rutgers

The Eagles' two leading rushers are their starting running back Miles Sanders and their quarterback Jalen Hurts.
This is just a subset of Hurts' overall rushing performance, since he often "scrambles" on passing plays, which count as rushing yards even though they are the result of passing plays.

Is this the end of the story? Give the time on the clock remaining for this live training, you can hopefully guess the answer.

## Rushing Yards Over Expected: Building linear model

The winning solution to the 2020 [Big Data Bowl](https://operations.nfl.com/gameday/analytics/big-data-bowl/), an analytics competition held by the NFL each year, resulted in the construction of an expectation metric for rushing yardage that depended on a number of factors.
From there, one can look at the residual between actual outcome an expected outcome - called rushing yards over expected (RYOE) for more insight into how well the rusher did relative to expectations.

Here, we fit a simple linear model based only on the number of yards to go for a first down.
Generally speaking, defenses play closer to the line of scrimmage when there are fewer yards to go, making rushing yardage harder to gain the closer `ydstogo` gets to 0.

* Use the `smf.ols()` function with `rushing_yards` predicted by an intercept and `ydstogo`.
* Use the input data `pbp_run`
* Save the output as `yard_to_go`

In [18]:
# use the smf.ols() with a formula and data


Next, extract the `fit()` and then `print()` the `summary()`:

In [19]:
# extract fit and then print() the summary

Notice that the coefficient for `ydstogo` is positive - a rusher is expected to gain more yards the further his team is from the first down market.
Let's explore the outcome of this model.

Lastly, create a new column, _run yards over expected_ or `ryoe` in `pbp_run` from `yard_to_go.fit().resid`:

In [20]:
# create ryoe column in pbp_run from the resid of yard_to_go.fit().resid


### RYOE for Super Bowl Rushers

First, you will need to filter out only the two teams in the Super Bowl.
You will results the object `sb_teams` from before, so you may need to go up and load this if you're jumping into the notebook now.
To do the filter, use the same `isin()` function as before (the new data frame is called pbp_ryoe_sb to be different from the previously data frame created above):

* Use `loc[]` with `pbp_run`
* use `isin()` with `sb_teams` and the `posteam` column from `pbp_run`
* Be sure to `reset_index()`
* Save the new data frame as `pbp_ryoe_sb`

In [21]:
# filter out sb teams to create pbp_ryoe_sb

Next, use the same grouping used before to calculate each player's rushing yards (sum of rushing_yards), yards-per-carry (mean of rushing yards), total rushing ards over expected (TYOE; sum of RYOE), and average RYOE (mean of RYOE):

In [22]:
# 1. create pbp_ryoe_sb_yards
# 2. use pbp_ryoe_sb
# 3. group by ["posteam", "rusher_player_id", "rusher_player_name"]
# 4. aggregate with "rushing_yards": ["sum", "mean"]
#    as well as "ryoe": ["sum", "mean", "count"]


Let's look at the play-level distributions of these players for guys with more than 20 carries during the course of the season.
We include the cleaning code, because, while we spend most of our time cleaning data, we do not want this training to be a `pandas` session on cleaning data.

We used the value 20 because we only want players who had a large number of players:
First, clean up the column names to be easier to work with:

In [28]:
# first, flatten the columns of pbp_ryoe_sb_yards


# Then rename columns


# Query for carries >= 20


# Sort values by TYOE, ascending=False


Notice that while Pacheco was close to Sanders in terms of yards per carry, and trailed him in total rushing yards, he is pretty close to him in total RYOE and is much better than he is in average RYOE per carry. 

Now, let's look at the data.
First, we need to extract the top players for the superbowl.
To do this,

1. Create `pbp_run_sb_20`
2. Use the `pbp_run` data
3. Filter using `.loc[]` with `pbp_run['rusher_player_id']` by `.isin(pbp_ryoe_sb_yards_20.rusher_player_id.unique())`

In [24]:
# Filter out only players with more than 20 carries


We'll use a `seaborn` boxplot for each player:

In [25]:
# Use sns boxplot
# x is ryoe, y is rusher_player_name, hue is posteam, data is pbp_run_sb_20


Notice that, while Jerick McKinnon has scored 9 touchdowns for the Chiefs this year, he's actually struggled running the ball relative to expectations pre-snap.
Former first-round pick Clyde Edwards-Helaire has ails well, with a negative RYOE, which opened the door for seventh-round pick Pacheco to shine.

### Conclusions for simple model

What have we learned from this simple modeling exercise?

* Rushing yards over expected is right skewed! 
* Sanders and Pacheco have shined, while Scott, Edwards-Helaire and McKinnon have struggled
* Much of data science, even with "package" data still requires data wrangling!

While anything can happen in Sunday's game, watch out for Sanders and Pacheco.

Lastly, check out our forthcoming book, [Football Analytics with Python & R](https://learning.oreilly.com/library/view/football-analytics-with/9781492099611/]) or our DataCamp courses to learn about [Linear Algebra in R](https://app.datacamp.com/learn/courses/linear-algebra-for-data-science-in-r), [Hierarchical and Mixed-effect Models in R](https://app.datacamp.com/learn/courses/hierarchical-and-mixed-effects-models-in-r), or [Generalized Linear Models in R](https://app.datacamp.com/learn/courses/generalized-linear-models-in-r).
And feel free to follow Eric on [Twitter](https://twitter.com/ericeager__) or [LinkedIn](https://www.linkedin.com/in/eric-eager-31a2a220) and Richard on [LinkedIn](https://www.linkedin.com/in/raerickson).

## Q and A

Questions?

### Multiple Regression

But, what about a more complex model? Can you do better at predicting?
As a bonus, we include the code for a more complicated model.

We have more powerful tools in our toolbox compared to a simple linear regression.
A next logical step would be to use multiple predictor variables and use a multiple regression.
There are other things that go into expected rushing success.
For example, which down it is?, how many yards the team has to go to reach the end zone?, what is the run direction?, what is the score differential? how much time is remaining?

Depending upon the purpose of the extra variables, you might say you are _correcting_ for other predictors (a term commonly used in the medical field, for example, when Richie Cotton worked at a Medical College he would have used the term).
Or, we may want to simply include multiple predictors.
However, this added realism comes at a trade-off of complexity.

If you are joining us late, please make sure you load the data from the previous part of the tutorial.
First, reformat `down` to be a string.
This will cause Python's regression to treat `down` as a category, rather than an a continuous predictor:

In [29]:
# Convert down from pbp_run to be a string using astype(str)


Next, let's build a model and examine the fit's summary.
Because the formula is long, define it first to make your code easier to read:

In [30]:
# include "rushing_yards ~ 1 + ydstogo + down + yardline_100 + run_location + score_differential + game_seconds_remaining"


In [31]:
# next build the model, extract the fit, and print the summary


Now, extract out RYOE, similar to and calculate the same summaries as before:

In [32]:
# extract out the residual from model 1


# use .loc to extract out the sb_teams


# group by ["posteam", "rusher_player_id", "rusher_player_name"]
# aggregate with a dictionary
#    {"rushing_yards": ["sum", "mean"],
#     "ryoe": ["sum", "mean"],
#     "ryoe_m1": ["sum", "mean", "count"]})


# Filter to create top players with >20 plays data frame

# sort and print outputs

Next, filter for only players with more than 20 players.

In [33]:
                           ].isin(pbp_ryoe_sb_yards_20.reset_index().rusher_player_id.unique())]

The data requires one more manipulation, a melt, before the boxplot:

In [35]:
# melt 

In [36]:
# sns boxplot

Notice here that results are largely the same.

# Conclusion

In this training session we looked at real National Football League play-by-play data to gain insights into the running game for Super Bowl LVII.

You saw how a multiple regression produced similar, but slightly different results than the simple regression.
As a data scientist, which model would you use?
Conversely, if you had to explain the model as a football analysts on live TV or radio, which model would you use?